In [2]:
import pandas as pd

# Load data from CSV files
customers = pd.read_csv('customers.csv')
orders = pd.read_csv('orders.csv')
orderitems = pd.read_csv('orderitems.csv')
products = pd.read_csv('products.csv')

# Display the first few rows of each dataframe
print(customers.head())

print(orders.head())

print(orderitems.head())

print(products.head())

# Check for missing values
print(customers.isnull().sum())
print(orders.isnull().sum())
print(orderitems.isnull().sum())
print(products.isnull().sum())

# Handle missing values
# Example: Fill missing values or drop rows with missing values
customers.fillna('', inplace=True)
orders.dropna(inplace=True)
orderitems.dropna(inplace=True)
products.dropna(inplace=True)

# Ensure correct data types
orders['OrderDate'] = pd.to_datetime(orders['OrderDate'])


   CustomerID FirstName LastName                     Email     Phone  \
0           1      Amit   Sharma   amit.sharma@example.com  555-0101   
1           2    Rajesh    Kumar  rajesh.kumar@example.com  555-0102   
2           3    Vikram    Singh  vikram.singh@example.com  555-0103   
3           4    Sunita      Rai    sunita.rai@example.com  555-0104   
4           5     Anita    Gupta   anita.gupta@example.com  555-0105   

       Address   City  State  ZipCode  
0  123 Main St  Delhi  Delhi   110001  
1  124 Main St  Delhi  Delhi   110002  
2  125 Main St  Delhi  Delhi   110003  
3  126 Main St  Delhi  Delhi   110004  
4  127 Main St  Delhi  Delhi   110005  
   OrderID   OrderDate  CustomerID
0        1  2023-03-15          40
1        2  2023-07-11          17
2        3  2023-03-26           8
3        4  2023-09-29          27
4        5  2023-11-27          54
   OrderItemID  OrderID  ProductID  Quantity   Price
0            1      439         14         8   88.45
1          

In [3]:
# Merge dataframes to get a complete dataset
merged_df = pd.merge(orders, orderitems, on='OrderID')
merged_df = pd.merge(merged_df, customers, on='CustomerID')
merged_df = pd.merge(merged_df, products, on='ProductID')

# Calculate Recency, Frequency, and Monetary values

# Recency: Days since last purchase
recency_df = merged_df.groupby('CustomerID')['OrderDate'].max().reset_index()
recency_df['Recency'] = (pd.to_datetime('now') - recency_df['OrderDate']).dt.days

# Frequency: Number of orders
frequency_df = merged_df.groupby('CustomerID')['OrderID'].nunique().reset_index()
frequency_df.columns = ['CustomerID', 'Frequency']

# Monetary: Total amount spent
merged_df['TotalSpent'] = merged_df['Quantity'] * merged_df['Price']
monetary_df = merged_df.groupby('CustomerID')['TotalSpent'].sum().reset_index()
monetary_df.columns = ['CustomerID', 'Monetary']

# Merge RFM values into a single dataframe
rfm_df = pd.merge(recency_df[['CustomerID', 'Recency']], frequency_df, on='CustomerID')
rfm_df = pd.merge(rfm_df, monetary_df, on='CustomerID')

print(rfm_df.head())



   CustomerID  Recency  Frequency  Monetary
0           1      221          2    425.49
1           2      267          4   5118.36
2           3      394          1    216.40
3           4      262          3   2184.87
4           5      241          3   2007.19
